In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import scipy

E:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Feature Extraction of Training and Testing

In [2]:
filepath = 'C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/'
#####################   Modify above path   #########################

testpath=filepath+'/data/test_set/'
trainpath=filepath+'/data/train_set/'

test_image_dir = testpath + "images/"
test_pt_dir = testpath + "points/"
train_image_dir = trainpath + "images/"
train_pt_dir = trainpath + "points/"

In [3]:
import sklearn.metrics.pairwise
def pairwise_dist(vec):
    dist  = sklearn.metrics.pairwise_distances(vec, metric='euclidean')
    np.fill_diagonal(dist, np.nan)
    return dist
def feature(fiducial_pt_list,index):
    pairwise_dist_feature = pairwise_dist(fiducial_pt_list[index]).flatten()
    pairwise_dist_feature = pairwise_dist_feature[~np.isnan(pairwise_dist_feature)]
    return pairwise_dist_feature

# Training Feature

In [4]:
dataDir = train_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


X = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    X.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)

In [5]:
y =pd.read_csv(trainpath+'label.csv')['emotion_idx']

In [6]:
X

,0,1,2,3,4,5,6,7,8,9,...,5996,5997,5998,5999,6000,6001,6002,6003,6004,6005
0,46.0,30.0,14.0,19.0,37.0,25.0,16.0,29.0,176.0,136.0,...,414.0,398.0,381.0,359.0,318.0,274.0,228.0,171.0,114.0,57.0
1,45.0,27.0,11.0,21.0,37.0,29.0,24.0,32.0,170.0,136.0,...,406.0,395.0,384.0,364.0,329.0,281.0,231.0,175.0,117.0,59.0
2,42.0,24.0,11.0,19.0,34.0,23.0,20.0,26.0,153.0,119.0,...,374.0,363.0,357.0,340.0,305.0,259.0,213.0,162.0,109.0,54.0
3,47.0,29.0,15.0,18.0,31.0,17.0,14.0,31.0,166.0,126.0,...,422.0,409.0,398.0,378.0,337.0,286.0,237.0,183.0,122.0,61.0
4,37.0,22.0,6.0,19.0,36.0,24.0,18.0,26.0,161.0,122.0,...,383.0,377.0,364.0,344.0,308.0,263.0,217.0,164.0,109.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,36.0,20.0,14.0,23.0,36.0,19.0,6.0,23.0,153.0,114.0,...,343.0,331.0,318.0,300.0,272.0,235.0,198.0,153.0,104.0,54.0
2496,32.0,15.0,7.0,18.0,34.0,19.0,8.0,12.0,128.0,100.0,...,294.0,287.0,278.0,265.0,246.0,219.0,185.0,144.0,91.0,47.0
2497,30.0,17.0,9.0,14.0,26.0,16.0,14.0,20.0,128.0,101.0,...,323.0,321.0,310.0,290.0,254.0,217.0,170.0,131.0,89.0,45.0
2498,30.0,17.0,9.0,23.0,41.0,24.0,12.0,18.0,138.0,106.0,...,296.0,300.0,298.0,285.0,264.0,232.0,199.0,157.0,101.0,47.0


In [7]:
y

0        1
1        1
2        1
3        1
4        1
        ..
2495    22
2496    22
2497    22
2498    22
2499    22
Name: emotion_idx, Length: 2500, dtype: int64

# Testing Feature

In [8]:
dataDir = test_pt_dir
fiducial_pt_list = []
filelist = []
for file in os.listdir(dataDir):
    filelist.append(file)
filelist.sort()
for file in filelist:
    fiducial_pt_list.append(scipy.io.loadmat(dataDir+file))
    l = []
for i in range(len(fiducial_pt_list)):
    if 'faceCoordinatesUnwarped' in fiducial_pt_list[i].keys():
        l.append(fiducial_pt_list[i]['faceCoordinatesUnwarped'])
    else:
        l.append(fiducial_pt_list[i]['faceCoordinates2'])
        
fiducial_pt_list = l


TestInput = pd.DataFrame(np.zeros((2500, 6006)))
for i in range(2500):
    TestInput.iloc[i,:] = np.round(feature(fiducial_pt_list, i).flatten(), 0)

In [9]:
TestInput

,0,1,2,3,4,5,6,7,8,9,...,5996,5997,5998,5999,6000,6001,6002,6003,6004,6005
0,46.0,30.0,14.0,19.0,37.0,25.0,16.0,29.0,176.0,136.0,...,414.0,398.0,381.0,359.0,318.0,274.0,228.0,171.0,114.0,57.0
1,45.0,27.0,11.0,21.0,37.0,29.0,24.0,32.0,170.0,136.0,...,406.0,395.0,384.0,364.0,329.0,281.0,231.0,175.0,117.0,59.0
2,42.0,24.0,11.0,19.0,34.0,23.0,20.0,26.0,153.0,119.0,...,374.0,363.0,357.0,340.0,305.0,259.0,213.0,162.0,109.0,54.0
3,47.0,29.0,15.0,18.0,31.0,17.0,14.0,31.0,166.0,126.0,...,422.0,409.0,398.0,378.0,337.0,286.0,237.0,183.0,122.0,61.0
4,37.0,22.0,6.0,19.0,36.0,24.0,18.0,26.0,161.0,122.0,...,383.0,377.0,364.0,344.0,308.0,263.0,217.0,164.0,109.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,36.0,20.0,14.0,23.0,36.0,19.0,6.0,23.0,153.0,114.0,...,343.0,331.0,318.0,300.0,272.0,235.0,198.0,153.0,104.0,54.0
2496,32.0,15.0,7.0,18.0,34.0,19.0,8.0,12.0,128.0,100.0,...,294.0,287.0,278.0,265.0,246.0,219.0,185.0,144.0,91.0,47.0
2497,30.0,17.0,9.0,14.0,26.0,16.0,14.0,20.0,128.0,101.0,...,323.0,321.0,310.0,290.0,254.0,217.0,170.0,131.0,89.0,45.0
2498,30.0,17.0,9.0,23.0,41.0,24.0,12.0,18.0,138.0,106.0,...,296.0,300.0,298.0,285.0,264.0,232.0,199.0,157.0,101.0,47.0


In [10]:
TestInput.columns=X.columns

In [11]:
scaler = StandardScaler()
TestInput = scaler.fit_transform(TestInput)

In [12]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

train_x_dis,test_x_dis,train_y_dis,test_y_dis=train_test_split(X,y,test_size=0.2,random_state=3662)

# Model Training

## BaselineModel: GBM

In [13]:
Baseline_model= GradientBoostingClassifier(n_estimators=100  , max_depth= 1,learning_rate=0.1)

In [14]:
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(Baseline_model, X,y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 40.24% (2.24%)


In [15]:
Baseline_model.fit(X,y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=1,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [16]:
joblib.dump(Baseline_model,'BASELINE_model.m')#please change the path

['BASELINE_model.m']

## Advanced Model: VotingClassifier

In [17]:
randomforest_model_final=RandomForestClassifier(n_estimators = 100, criterion = 'gini', 
                             random_state = 42, min_samples_leaf=1, max_features='sqrt')

In [18]:
xgboost_model_final =xgboost.XGBClassifier(max_depth=4,n_estimators=50,learning_rate=0.1,
                       min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,reg_alpha=0.005)

In [19]:
svm_model_final =SVC(C=0.1,decision_function_shape='ovr',degree=2,gamma=0.1,kernel='linear')

In [20]:
logistic_model_final =LogisticRegression(C=0.01,
                                         dual=False,
                                         fit_intercept=True,
                                         intercept_scaling=1,
                                         max_iter=300,
                                         multi_class='multinomial',
                                         penalty='l2',
                                         solver='newton-cg',
                                         tol=0.0001)

In [21]:
MLP_model_final =MLPClassifier(early_stopping=True,
                              hidden_layer_sizes=(3000,),
                              learning_rate='adaptive',
                              solver='lbfgs',
                              validation_fraction=0.2)

In [22]:
Final_model = VotingClassifier(
    estimators=[('rf_clf',randomforest_model_final),
                ('log_clf', logistic_model_final),
                ('svm_clf', svm_model_final),
               ('xgb_clf', xgboost_model_final),
                ('MLP_clf',MLP_model_final)],
    voting='hard',
#    weights=[0.45,0.54,0.50,0.47,0.49],
    
)

In [23]:
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(Final_model, X,y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 53.96% (2.52%)


In [24]:
Final_model.fit(X,y)

VotingClassifier(estimators=[('rf_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='sqrt',
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     n_estimators=100,
                                                     n_jobs=None,
      

In [25]:
joblib.dump(Final_model,'FINAL_model.m')#please change the path

['FINAL_model.m']

# Prediction

In [33]:
# joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/FINAL_model.m')
FINAL_model=joblib.load('FINAL_model.m')

# joblib.load('C:/Users/jiaoh/Documents/GitHub/Spring2020-Project3-group2/hanbojiao_test/doc/BASELINE_model.m')
BASELINE_model=joblib.load('BASELINE_model.m')

In [27]:
predict_Advanced=FINAL_model.predict(TestInput)
predict_Baseline=BASELINE_model.predict(TestInput)

In [28]:
label_prediction = pd.read_csv(filepath+'data/labels_prediction.csv')
label_prediction['Baseline']=predict_Baseline
label_prediction['Advanced']=predict_Advanced
label_prediction.to_csv(filepath+'data/labels_prediction.csv')

In [29]:
FINAL_model.score(TestInput,y)

1.0

In [30]:
FINAL_model.score(X,y)

1.0

In [31]:
BASELINE_model.score(TestInput,y)

0.74

In [32]:
BASELINE_model.score(X,y)

0.74